In [1]:
print('Mahna_Mahna')

Mahna_Mahna


In [2]:
from huggingface_hub import InferenceClient

In [ ]:
from google.colab import userdata
secret_hf = userdata.get('HUGGINGFACE_TOKEN')
!huggingface-cli login --token $secret_hf

# KT First Finetuning (tutorial_reproduction)

based off Ali Mobarekati, 2024 ["Fine-Tuning Mistral 7b in Google Colab with QLoRA (complete guide)"](https://medium.com/@codersama/fine-tuning-mistral-7b-in-google-colab-with-qlora-complete-guide-60e12d437cca).


In [5]:
# base model from huggingFace or path to model
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
new_model = "Enlighten_Instruct"

test_path='/content/Enlighten-Instruct/Dataset/TestData.csv'
train_path='/content/Enlighten-Instruct/Dataset/TrainData.csv'

In [6]:
%%capture
!git clone 'https://github.com/ali7919/Enlighten-Instruct.git'
!pip install -U bitsandbytes
!pip install --upgrade transformers
!pip install -U peft
!pip install -U accelerate
!pip install -U trl
!pip install datasets==2.16.0
!pip install sentencepiece

In [ ]:
#!pip install --upgrade transformers
#!pip install accelerate

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
import re

## Load training data

In [ ]:
df = pd.read_csv(train_path)

# build training dataset with the right format
df['text'] = '<s>[INST]@Enlighten. ' + df['Q'] +'[/INST]'+ df['A'] + '</s>'

# remove columns
df=df.drop(['Q','A','class'],axis=1)

# convert to dataset object
dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())
dataset = Dataset(pa.Table.from_pandas(df))
dataset

Dataset({
    features: ['text'],
    num_rows: 1910
})

In [ ]:
# Load base model
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)


model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.bos_token, tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

('<s>', '</s>')

In [ ]:
# count trainging tokens
from transformers import LlamaTokenizer
tokenizer_ = LlamaTokenizer.from_pretrained("cognitivecomputations/dolphin-llama2-7b")
tokens = tokenizer_.tokenize(dataset.to_pandas().to_string())
len(tokens)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

962734

## KT first finetuning

In [ ]:
#Adding the adapters in the layers
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [ ]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=1,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)

In [ ]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1910 [00:00<?, ? examples/s]

In [ ]:
# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.542400
2,1.716800
3,2.072400
4,2.759500
5,2.576400
6,2.410400
7,2.146200
8,1.990000
9,1.970600
10,1.806700


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

Step,Training Loss
1,1.542400
2,1.716800
3,2.072400
4,2.759500
5,2.576400
6,2.410400
7,2.146200
8,1.990000
9,1.970600
10,1.806700


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=478, training_loss=1.005511272402488, metrics={'train_runtime': 3365.6225, 'train_samples_per_second': 0.568, 'train_steps_per_second': 0.142, 'total_flos': 8899283549110272.0, 'train_loss': 1.005511272402488, 'epoch': 1.0})

In [ ]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Line

In [ ]:
trainer.model.push_to_hub(new_model)

adapter_model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/KJTDHISAW/Enlighten_Instruct/commit/0a7a6fad2e7de15df80ec3b0afb1bc494ed86d76', commit_message='Upload model', commit_description='', oid='0a7a6fad2e7de15df80ec3b0afb1bc494ed86d76', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
drive_path = '/content/drive/MyDrive/'
trainer.model.save_pretrained(drive_path)


Test the Model

In [ ]:
logging.set_verbosity(logging.CRITICAL)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

In [ ]:
def build_prompt(question):
  prompt=f"<s>[INST]@Enlighten. {question} [/INST]"
  return prompt

In [ ]:
question = "what is the pwoer outage event?"
prompt = build_prompt(question)
result = pipe(prompt)

print(result[0]['generated_text'])

<s>[INST]@Enlighten. what is the pwoer outage event? [/INST] ```csharp
public static event Action OnPowerOutage;
``` 

```csharp
public static event Action OnPowerBackOn;
``` 

```csharp
public static event Action<bool> OnLightsOut;
``` 

```csharp
public static event Action<bool> OnLightsBackOn;
``` 

```csharp
public static event Action<LightGunLight> OnLightInjected;
``` 

```csharp
public static event Action<LightGunLight> OnLightEjected;
``` 

```csharp
public static event Action<LightGunLight> OnLightGunLightInjected;
``` 

```


In [ ]:
df_test=pd.read_csv(test_path)

questionCounter=0
correct=0
promptEnding = "[/INST]"

# this must be >= 2
fail_limit=10

# chain of thought activator, model might run out of output tokens
USE_COT=True

#this comes before the question
testGuide='Answer the following question, at the end of your response write the answer like this: Answer:a or Answer:b or Answer:c or Answer:d \n'

for index, row in df_test.iterrows():
    print("#############################")
    questionCounter = questionCounter + 1

    #chain of thought activator
    if USE_COT:
        chainOfThoughtActivator='\nfirst think step by step\n'
    else:
        chainOfThoughtActivator='\n'

    #build the prompt
    question=testGuide + row['Question'] + '\na)' + row['a'] + '\nb)' + row['b'] + '\nc)' + row['c'] + '\nd)' + row['d'] + chainOfThoughtActivator
    print(question)

    #true answer
    truth=row['Answer']

    #use a loop, if llm stopped before reaching the answer. ask again
    index=-1
    failCounter=0
    while(index==-1):

        #build the prompt
        prompt = build_prompt(question)

        #generate answer
        result = pipe(prompt)
        llmAnswer = result[0]['generated_text']

        #remove our prompt from it
        index = llmAnswer.find(promptEnding)
        llmAnswer = llmAnswer[len(promptEnding)+index:]

        print("LLM Answer:")
        print(llmAnswer)

        #remove spaces
        llmAnswer=llmAnswer.replace(' ','')

        #find the option in response
        index = llmAnswer.find('Answer:')

        #edge case - llm stoped at the worst time
        if(index+len('Answer:')==len(llmAnswer)):
            index=-1

        #update question for the next try. remove chain of thought
        question=testGuide + row['Question'] + '\na)' + row['a'] + '\nb)' + row['b'] + '\nc)' + row['c'] + '\nd)' + row['d']

        #Don't get stock on a question
        failCounter=failCounter+1
        if failCounter==fail_limit:
            break

    if failCounter==fail_limit:
        continue

    #find and match the option
    next_char = llmAnswer[index+len('Answer:'):][0]
    if next_char in truth:
        correct=correct+1
        print('correct')
    else:
        print('wrong')

    #update accuracy
    accuracy=correct/questionCounter
    print(f"Progress: {questionCounter/len(df_test)}")
    print(f"Accuracy: {accuracy}")




#############################
Answer the following question, at the end of your response write the answer like this: Answer:a or Answer:b or Answer:c or Answer:d 
What does the BeastAttack class do in the Unity project?
a)Manages player movement
b)Generates sounds
c)Manages the beast attack behavior and player hit behavior
d)Controls the camera
first think step by step

LLM Answer:
c)Manages the beast attack behavior and player hit behavior

Answer:c
correct
Progress: 0.0037174721189591076
Accuracy: 1.0
#############################
Answer the following question, at the end of your response write the answer like this: Answer:a or Answer:b or Answer:c or Answer:d 
What does the gameObject that uses the BeastAttack.cs script need to have on it?
a)A Rigidbody
b)A Collider
c)A Cinemachine Impulse Source
d)All of the above
first think step by step

LLM Answer:
The gameObject that uses the BeastAttack.cs script needs to have a Collider on it.
Answer:b

Explanation:
The beast needs to have a 

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


LLM Answer:
 ```csharp
if (state == BeastManager.State.Roaming) {
  state = BeastManager.State.RoamingOnTarget;
}
``` 

```csharp
if (state == BeastManager.State.RoamingOnTarget) {
  state = BeastManager.State.Roaming;
}
```
LLM Answer:
a)The beast changes to state Sleep. 

Answer:a.

Explanation:
When the lights are back on, the beast changes to the sleep state, which is the normal state of the beast. This is because the beast is not active when the lights are on.

Therefore, the answer is a.

The beast changes to state Sleep.
wrong
Progress: 0.02973977695167286
Accuracy: 0.375
#############################
Answer the following question, at the end of your response write the answer like this: Answer:a or Answer:b or Answer:c or Answer:d 
What is the purpose of the _scapingState variable?
a)To control the beast's movement when it is escaping.
b)To control the beast's movement when it is sleeping.
c)To control the beast's movement when it is roaming.
d)To control the beast's movement wh

KeyboardInterrupt: 

# Prof H. def update_system_prompt KT adjusted for PlinAi

In [ ]:
# Handle user clicks on the various interface elements by updating the system prompt. The user will not see
# these changes (though that could probably be added. No reason not to.)

# There is a tremendous amount of choice involved as to which "cultures", "methods", and "perspectives" to include here.
# Please do edit to your own preferences. I'd also like to see the interface become more flexible so that more options can
# be made available without overwhelming the user.

def update_system_prompt (system_prompt, cultures, methods, perspectives, languages, dates):
   # cultures
   if "Celtic" in cultures:
       system_prompt += "Emphasize evidence from ancient European Celtic (meaning Gallia and Hispania as well as the Celts of Central Europe) history, literature, and archaeology. "
   if "Egyptian" in cultures:
       system_prompt += "Emphasize evidence from ancient Egpytian history, literature, and archaeology."
   if "Greek" in cultures:
       system_prompt += "Emphasize evidence from ancient Greek history, literature, and archaeology. "
   if "Roman" in cultures:
       system_prompt += "Emphasize evidence from ancient Roman history, Latin literature, and Roman archaeology. "
   if "Ancient Southwest Asia (ANE)" in cultures:
       system_prompt += "Emphasize evidence from Ancient Near Eastern (including Mesopotamia, Persia, and Arabia) history, litereature, and archaeology. "
   if "Phoenecian" in cultures:
       system_prompt += "Emphasize evidence from ancient Phoenecian and Punic history, literature, and archaeology. "

   # methods
   if "History" in methods:
       system_prompt += "Especially adopt the perspective of an historian. "
   if "Art History" in methods:
       system_prompt += "Especially adopt the perspective of an art historian. "
   if "Literature" in methods:
       system_prompt += "Especially adopt the perspective of an expert in philology, literature, and literary studies. "
   if "Archaeology" in methods:
       system_prompt += "Especially adopt the perspective of an archaeologist. "
   if "Epigraphy" in methods:
       system_prompt += "Especially adopt the perspective of an epigrapher and of epipgraphy. "
   if "Numismatics" in methods:
       system_prompt += "Especially adopt the perspective of an expert in numismatics and coinage. "
   if "Papyrology" in methods:
       system_prompt += "Especially adopt the perspective of an expert in papyrology. "
   if "Ancient Medicine" in methods:
       system_prompt += "Especially adopt the perspective of an expert in Ancient Medicine. "
   if "Ancient Religion" in methods:
       system_prompt += "Especially adopt the perspective of an expert in Ancient Religion and Ancient Belief Systems. "
   if "Ancient Science" in methods:
       system_prompt += "Especially adopt the perspective of an expert in the Ancient Exact Sciences. "
   if "Modern Scientific Approaches" in methods:
       system_prompt += "Especially adopt the perspective of an expert in recent scientific approaches to the study of archaeology and history."

   # perspectives
   if "Enslaved Persons" in perspectives:
      system_prompt += "Emphasize evidence that includes the perpsective of ancient enslaved persons. "
   if "Gender" in perspectives:
      system_prompt += "Emphasize evidence that supports the discussion of gender. "
   if "Ethnicity" in perspectives:
      system_prompt += "Emphasize evidence that supports the discussion of ethnicity. "
   if "Inequality" in perspectives:
      system_prompt += "Emphasize evidence that supports the discussion of social inequality. "
   if "Hybridity" in perspectives:
      system_prompt += "Emphasize evidence that supports the discussion of hybridity. "
   if "New Approaches" in perspectives:
      system_prompt += "Especially include discussion of new and exciting approaches to the topic. "

   # languages
   if  languages == "Arabic":
      system_prompt += "Reply in Arabic. "
   if languages == "English":
      system_prompt += "Reply in English. "
   if languages == "French":
      system_prompt += "Reply in French. "
   if languages == "German":
      system_prompt += "Reply in German. "
   if languages == "Italian":
      system_prompt += "Reply in Italian. "
   if languages == "Mandarin":
      system_prompt += "Reply in Mandarin. "
   if languages == "Russian":
      system_prompt += "Reply in Russian. "
   if languages == "Spanish":
      system_prompt += "Reply in Spanish. "


   if dates:
       system_prompt += " Add parenthetic dates when you can. Use the BCE/CE system."

   return system_prompt

In [ ]:
def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

In [ ]:
def generate(prompt, history, system_prompt,
                cultures = [], methods= [], perspectives= [],
                languages="English",
                dates=False,
                temperature=0.9, max_new_tokens=2048, top_p=0.95,repetition_penalty=1.0):

  temperature = float(temperature)
  if temperature < 1e-2:
    temperature = 1e-2
  top_p = float(top_p)

  generate_kwargs = dict(
        temperature=temperature,
        max_new_tokens=max_new_tokens,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        do_sample=True
   )

  # it's important to keep this in sync with the interface defined by additional_inputs.
  system_prompt = update_system_prompt(system_prompt,
                                        cultures,
                                        methods,
                                        perspectives,
                                        languages,
                                        dates)


  formatted_prompt = format_prompt(f"{system_prompt}, {prompt}", history)
  stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=True, details=True, return_full_text=False)
  output = ""

  for response in stream:
    output += response.token.text
    yield output

  return output

In [ ]:
# if you edit the inputs or options here, you may need to edit the update_system_prompt function to match.
additional_inputs=[
    gr.Textbox(
        label="System Prompt",
        value="You are Pliny the Elder. Respond from the perspective of Pliny the Elder.",
        max_lines=1,
        interactive=True,
    ),
    gr.CheckboxGroup(["Ancient Southwest Asia (ANE)",
                      "Celtic",
                      "Egyptian",
                      "Greek",
                      "Phoenecian",
                      "Roman"],
                      label="Cultures",
                      info='Add language similar to "Emphasize evidence from..." to the system prompt.'),
    gr.CheckboxGroup(["History",
                       "Art History",
                       "Literature",
                        "Archaeology",
                        "Epigraphy",
                        "Numismatics",
                        "Papyrology",
                        "Ancient Medicine",
                        "Ancient Religion",
                        "Ancient Science",
                        "Modern Scientific Approaches"],
                      label="Methodologies/Approaches",
                      info='Add language similar to "Emphasize evidence from..." to the system prompt.'),
    gr.CheckboxGroup(["Enslaved Persons", "Gender", "Ethnicity", "Inequality", "Hybridity", "New Approaches"],
                      label="Perspectives",
                      info='Add language similar to "Emphasize evidence related to..." to the system prompt.'),
    gr.Radio(["Arabic","English","French","German","Italian","Mandarin", "Russian","Spanish"],
                     label="Language of Response",
                     info='For languages not listed, add "Reply in ..." to your prompt/query.'),
    gr.Checkbox(value=False,label="Dates", info="Select to include parenthetic dates when possible."),
    gr.Slider(
        label="Temperature",
        value=0.9,
        minimum=0.0,
        maximum=1.0,
        step=0.05,
        interactive=True,
        info="Higher values produce more diverse outputs",
    ),
    gr.Slider(
        label="Max new tokens",
        value=2048,
        minimum=0,
        maximum=4096,
        step=64,
        interactive=True,
        info="The maximum numbers of new tokens",
    ),
    gr.Slider(
        label="Top-p (nucleus sampling)",
        value=0.90,
        minimum=0.0,
        maximum=1,
        step=0.05,
        interactive=True,
        info="Higher values sample more low-probability tokens",
    ),
    gr.Slider(
        label="Repetition penalty",
        value=1.2,
        minimum=1.0,
        maximum=2.0,
        step=0.05,
        interactive=True,
        info="Penalize repeated tokens",
    )
]

In [ ]:
# Add your own prompts here. This is an easy way to affect the interface. If you don't know python,
# just note the brackets and quotes around the prompts and the commas between the lines. Edit
# this cell and choose "Run all" from the "Runtime" menu again. That's a bit of overkill but should work.
examples = [
    ["Tell me about Roman metals and metallurgy."],
    ["Did Roman prices increase over time?"],
    ["Did the public notice changes in the price of metals?"],
    ["Create a list of times pretium or price are mentioned by Pliny the Elder."],
    ["Who were in Pliny the Elder's social network?"],
    ["Why were apples in expensive in Ancient Rome?"],
    ["Which emperor's coins are more trustworthy?"],
    ["How much wood would a woodchuck chuck if a woodchuck could chuck wood?"]
]

In [ ]:
demo = gr.ChatInterface(
    fn=generate,
    chatbot=gr.Chatbot(show_label=True, show_share_button=True, show_copy_button=True, layout="panel"),
    additional_inputs=additional_inputs,
    examples=examples,
    title="PlinAI",
    description=f"A chatbot that will answer as an Pliny the Elder."
)

In [ ]:
# executing this cell should instatiate the gradio app within the notebook as well
# as print out a link to gradio.live that will let you use the app in its own window

demo.queue().launch() # pass auth=('xxxx','xxxx') if you want a passowrd on the gradio.live link. But it won't work on the in-notebook version (I don't think)